# GPU  
在PyTorch中，每个数组都有一个设备（device），我们通常将其称为环境（context）。默认情况下，所有变量和相关的计算都分配给CPU。有时环境可能是GPU。  

在带有GPU的服务器上训练神经网络时，我们通常希望模型的参数在GPU上。  

### 1. 计算设备  
我们可以指定用于存储和计算的设备，如CPU和GPU。默认情况下，张量是在内存中创建的，然后使用CPU计算它。

在PyTorch中， CPU和GPU可以用`torch.device('cpu')` 和`torch.device('cuda')`表示。应该注意的是，cpu设备意味着所有物理CPU和内存，这意味着PyTorch的计算将尝试使用所有CPU核心。然而，gpu设备只代表一个卡和相应的显存。如果有多个GPU，我们使用`torch.device(f'cuda:{i}')` 来表示第i块GPU（i从0开始）。

In [6]:
import torch
from torch import nn
from d2l import torch as d2l

print(torch.device('cpu'))
print(torch.device('cuda:0'))
print(torch.cuda.device_count()) # 查询可用gpu的数量

cpu
cuda:0
1


### 2. 张量与GPU  

In [7]:
# 我们可以查询张量所在的设备。默认情况下，张量是在CPU上创建的。
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [9]:
# 存储在GPU上
X = torch.ones(2, 3, device=torch.device('cuda:0'))
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

不在同一GPU上的tensor如何进行计算？ 
 
![GPU复制](picture\GPU_copy.jpg)

In [ ]:
"""
Z = X.cuda(1)
print(X)
print(Z)

output:
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')
"""

### 3. 神经网络与GPU  
将模型参数放到GPU上。  
当输入为GPU上的张量时，模型将在同一GPU上计算结果。

In [10]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=torch.device('cuda:0'))

print(net(X))
print(net[0].weight.data.device)

tensor([[0.0319],
        [0.0319]], device='cuda:0', grad_fn=<AddmmBackward0>)
cuda:0


GPU在大数据情况下优势明显。  

In [ ]:
X_cpu = torch.randn(30000, 20000, dtype=torch.float32)
Y_cpu = torch.randn(20000, 2000, dtype=torch.float32)
X_gpu = torch.randn(30000, 20000, dtype=torch.float32, device=torch.device('cuda:0'))
Y_gpu = torch.randn(20000, 2000, dtype=torch.float32, device=torch.device('cuda:0'))
timer = d2l.Timer()
torch.matmul(X_cpu, Y_cpu)
time1 = timer.stop()
timer.start()
torch.matmul(X_gpu, Y_gpu)
time2 = timer.stop()
print(f'time1:{time1}, time2:{time2}')


time2: 6.851683616638184
